In [4]:
pip install -q langgraph langchain langchain-community transformers torch sentencepiece wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [7]:
import wikipedia
def search_wikipedia(topic:str)-> str :
  print(f"Searching Wikipedia for '{topic}'")
  try:
    page=wikipedia.page(topic,auto_suggest=False,redirect=True)
    return page.content
  except wikipedia.exceptions.PageError:
    return f"Error: Sorry I could not find a wikipedia page for '{topic}"
  except wikipedia.exceptions.DisambiguationError as e:
    return f'Error :{topic} us ambiguos . Did you mean one of these {e.options[:5]}'
  except Exception as e:
    return f'Error: {e}'

In [11]:
test_content=search_wikipedia("Reinforcement Learning")
print(f'\n output :{test_content[:200]}')

Searching Wikipedia for 'Reinforcement Learning'

 output :In machine learning and optimal control, reinforcement learning (RL) is concerned with how an intelligent agent should take actions in a dynamic environment in order to maximize a reward signal. Reinf


In [12]:
from transformers import T5ForConditionalGeneration , T5Tokenizer

model_name='t5-small'
try:
  tokenizer = T5Tokenizer.from_pretrained(model_name)
  model = T5ForConditionalGeneration.from_pretrained(model_name)
  print("Model and Tokenizer loaded successfully")
except Exception as e:
  print(f"Error loading model and tokenizer: {e}")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model and Tokenizer loaded successfully


In [19]:
def summarize_text(text:str)->str:
  if model is None:
    return 'Error : Summarization model is not available'
  print(f"Using Text Summarizer")

  text_to_summarize="summarize" + text
  inputs=tokenizer.encode(text_to_summarize,return_tensors='pt',max_length=512,truncation=True)
  summary_ids=model.generate(
      inputs,
      max_length=150,
      min_length=40,
      length_penalty=2.0,
      num_beams=4,
      early_stopping=True
  )

  summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
  return summary



In [20]:
class WebResearchAgent:
    def __init__(self):
        self.memory = {}

    def run(self, research_topic: str) -> str:
        print(f"\nAgent Starting Job for Topic: '{research_topic}'")

        print("\n[Thinking] I need to find information on the web for this topic.")
        print("[Thinking] I'll use the search_wikipedia tool")

        article_content = search_wikipedia(research_topic)
        self.memory["raw_article"] = article_content

        if "Error:" in article_content:
            print("[Observation] Search Tool Failed")
            return article_content
        else:
            print("[Observation] Search Tool Succeeded")

        print("\n[Thinking] The article is too long to read.")
        print("[Thinking] I'll use the summarization tool")

        summary = summarize_text(article_content)
        self.memory["summary"] = summary

        print("\nAgent Job finished")
        return summary

In [22]:
from IPython.display import display, Markdown
agent = WebResearchAgent()
research_topic = input("Enter your research topic: ")

summary = agent.run(research_topic)

display(Markdown(f"""
---

## Research Summary Report

### Topic
**{research_topic}**

### Summary
{summary}

---

*Generated by WebResearchAgent*
"""))

Enter your research topic: reinforcement Learning

Agent Starting Job for Topic: 'reinforcement Learning'

[Thinking] I need to find information on the web for this topic.
[Thinking] I'll use the search_wikipedia tool
Searching Wikipedia for 'reinforcement Learning'
[Observation] Search Tool Succeeded

[Thinking] The article is too long to read.
[Thinking] I'll use the summarization tool
Using Text Summarizer

Agent Job finished



---

## Research Summary Report

### Topic
**reinforcement Learning**

### Summary
reinforcement learning is one of the three basic machine learning paradigms. reinforcement learning involves training an agent through interactions with its environment. the agent makes decisions between trying new actions to learn more about the environment (exploration) or using current knowledge of the environment to take the best action (exploitation)

---

*Generated by WebResearchAgent*
